In [155]:
import pandas as pd
import json

In [183]:
"""

Travel Chat Bot for Slack
title:  amadeus_query.py
author: George Thoraninh
usage:  python amadeus_query.py

"""
# import statements
import os
import ssl
import requests
from functools import lru_cache

amadeus_api_key = os.environ['AMADEUS_API_KEY']
amadeus_api_secret = os.environ['AMADEUS_API_SECRET']


auth_response = requests.post('https://api.amadeus.com/v1/security/oauth2/token',
                              data={'grant_type': 'client_credentials',
                                    'client_id': amadeus_api_key,
                                    'client_secret': amadeus_api_secret})
bearer = auth_response.json()['access_token']
#print(bearer)
# API has quotas, so better to cache everything
@lru_cache(maxsize=2048)
def call_api(url, **params):
    full_url = f'https://api.amadeus.com/v1{url}'
    response = requests.get(full_url,
                            params=params,
                            headers={'Authorization': f'Bearer {bearer}',
                                     'Content-Type': 'application/vnd.amadeus+json'})
    return response.json()

response = call_api('/shopping/flight-offers',
                origin='YYZ',
                destination='JFK',
                departureDate='2020-08-01',
                returnDate='2020-08-15',
                adults='1',
                nonStop='true',
                currency='CAD',
                maxPrice='8000',
                max='2'
                )
print(response.keys())

dict_keys(['data', 'dictionaries', 'meta'])


In [184]:
df = pd.DataFrame(response['data'])

In [291]:
response['dictionaries']

{'carriers': {'DL': 'DELTA AIR LINES', 'WS': 'WESTJET'},
 'currencies': {'CAD': 'CANADIAN DOLLAR'},
 'aircraft': {'CR9': 'CANADAIR REGIONAL JET 900'},
 'locations': {'YYZ': {'subType': 'AIRPORT',
   'detailedName': 'LESTER B. PEARSON INTL'},
  'JFK': {'subType': 'AIRPORT', 'detailedName': 'JOHN F KENNEDY INTL'}}}

In [313]:
response['data']

[{'type': 'flight-offer',
  'id': '1586922740942-248136524',
  'offerItems': [{'services': [{'segments': [{'flightSegment': {'departure': {'iataCode': 'YYZ',
          'terminal': '3',
          'at': '2020-08-01T10:15:00-04:00'},
         'arrival': {'iataCode': 'JFK',
          'terminal': '4',
          'at': '2020-08-01T11:49:00-04:00'},
         'carrierCode': 'DL',
         'number': '4982',
         'aircraft': {'code': 'CR9'},
         'operating': {'number': '4982'},
         'duration': '0DT1H34M'},
        'pricingDetailPerAdult': {'travelClass': 'ECONOMY',
         'fareClass': 'K',
         'availability': 9,
         'fareBasis': 'KA0OA0MQ'}}]},
     {'segments': [{'flightSegment': {'departure': {'iataCode': 'JFK',
          'terminal': '4',
          'at': '2020-08-15T10:55:00-04:00'},
         'arrival': {'iataCode': 'YYZ',
          'terminal': '3',
          'at': '2020-08-15T12:36:00-04:00'},
         'carrierCode': 'DL',
         'number': '4983',
         'aircraft

In [186]:
# converts to string so we can get rid of square brackets []
str(df['offerItems'][0])[1:-1] 

"{'services': [{'segments': [{'flightSegment': {'departure': {'iataCode': 'YYZ', 'terminal': '3', 'at': '2020-08-01T10:15:00-04:00'}, 'arrival': {'iataCode': 'JFK', 'terminal': '4', 'at': '2020-08-01T11:49:00-04:00'}, 'carrierCode': 'DL', 'number': '4982', 'aircraft': {'code': 'CR9'}, 'operating': {'number': '4982'}, 'duration': '0DT1H34M'}, 'pricingDetailPerAdult': {'travelClass': 'ECONOMY', 'fareClass': 'K', 'availability': 9, 'fareBasis': 'KA0OA0MQ'}}]}, {'segments': [{'flightSegment': {'departure': {'iataCode': 'JFK', 'terminal': '4', 'at': '2020-08-15T10:55:00-04:00'}, 'arrival': {'iataCode': 'YYZ', 'terminal': '3', 'at': '2020-08-15T12:36:00-04:00'}, 'carrierCode': 'DL', 'number': '4983', 'aircraft': {'code': 'CR9'}, 'operating': {'number': '4983'}, 'duration': '0DT1H41M'}, 'pricingDetailPerAdult': {'travelClass': 'ECONOMY', 'fareClass': 'T', 'availability': 9, 'fareBasis': 'TAUNA0EL'}}]}], 'price': {'total': '360.96', 'totalTaxes': '91.97'}, 'pricePerAdult': {'total': '360.96', 

In [187]:
test = str(df['offerItems'][0])[1:-1] 

In [189]:
# needs to be in double quotes 
test2 = json.loads(test.replace('\'','\"'))

In [196]:
type(test2)

dict

In [191]:
# gets price of a ticket 
price = test2['price']['total']
price

'360.96'

In [193]:
test2.keys()

dict_keys(['services', 'price', 'pricePerAdult'])

In [200]:
df

,type,id,offerItems
0,flight-offer,1586922740942-248136524,[{'services': [{'segments': [{'flightSegment':...
1,flight-offer,1586922740942--1460165086,[{'services': [{'segments': [{'flightSegment':...


In [202]:
df[['offerItems']]

,offerItems
0,[{'services': [{'segments': [{'flightSegment':...
1,[{'services': [{'segments': [{'flightSegment':...


In [303]:
def json_parser(df, key):
    df = df[[key]] #takes data frame and dives deeper to nested JSON
    offer_json = str(df[key][0])[1:-1] # remove square brackets * only does one json for now
    return json.loads(offer_json.replace('\'','\"'))    

In [304]:
test2 = json_parser(df, 'offerItems')

In [305]:
test3 = str(test2['services'])[1:-1]
# test3 = json.loads(test3.replace('\'','\"'))  

In [306]:
# departing flight segment
listtest = [*test2['services'][0].values()] # used to unpack dict values

In [307]:
listtest_str = str(listtest)[2:-2] # got rid of these square brackets
listtest_dict = json.loads(listtest_str.replace('\'','\"'))    

In [308]:
listtest_dict.keys()

dict_keys(['flightSegment', 'pricingDetailPerAdult'])

In [309]:
# amount of tickets left
ticket_avail = listtest_dict['pricingDetailPerAdult']['availability']
ticket_avail

9

In [337]:
# depart time - needs parsing 
de_time = listtest_dict['flightSegment']['departure']['at']
de_time

'2020-08-01T10:15:00-04:00'

In [338]:
# return time - needs parsing 
re_time = listtest_dict['flightSegment']['arrival']['at']
re_time

'2020-08-01T11:49:00-04:00'

In [321]:
listtest_dict['flightSegment']['duration']

{'departure': {'iataCode': 'YYZ',
  'terminal': '3',
  'at': '2020-08-01T10:15:00-04:00'},
 'arrival': {'iataCode': 'JFK',
  'terminal': '4',
  'at': '2020-08-01T11:49:00-04:00'},
 'carrierCode': 'DL',
 'number': '4982',
 'aircraft': {'code': 'CR9'},
 'operating': {'number': '4982'},
 'duration': '0DT1H34M'}

In [341]:
# flight duration - needs parsing 
trip_duration = listtest_dict['flightSegment']['duration']
trip_duration

'0DT1H34M'

In [343]:
# flight carrier - needs parsing 
flight_carrier = listtest_dict['flightSegment']['carrierCode']
flight_carrier

'DL'

In [ ]:
listtest_dict['flightSegment']['carrierCode']

In [324]:
# returning flight segment
test2['services'][1]

{'segments': [{'flightSegment': {'departure': {'iataCode': 'JFK',
     'terminal': '4',
     'at': '2020-08-15T10:55:00-04:00'},
    'arrival': {'iataCode': 'YYZ',
     'terminal': '3',
     'at': '2020-08-15T12:36:00-04:00'},
    'carrierCode': 'DL',
    'number': '4983',
    'aircraft': {'code': 'CR9'},
    'operating': {'number': '4983'},
    'duration': '0DT1H41M'},
   'pricingDetailPerAdult': {'travelClass': 'ECONOMY',
    'fareClass': 'T',
    'availability': 9,
    'fareBasis': 'TAUNA0EL'}}]}

In [325]:
response['dictionaries']

{'carriers': {'DL': 'DELTA AIR LINES', 'WS': 'WESTJET'},
 'currencies': {'CAD': 'CANADIAN DOLLAR'},
 'aircraft': {'CR9': 'CANADAIR REGIONAL JET 900'},
 'locations': {'YYZ': {'subType': 'AIRPORT',
   'detailedName': 'LESTER B. PEARSON INTL'},
  'JFK': {'subType': 'AIRPORT', 'detailedName': 'JOHN F KENNEDY INTL'}}}

In [333]:
locations = [*response['dictionaries']['locations'].keys()]
locations

['YYZ', 'JFK']

In [344]:
response['dictionaries']['carriers']

{'DL': 'DELTA AIR LINES', 'WS': 'WESTJET'}

In [357]:
for key, value in response['dictionaries']['carriers'].items():
    if flight_carrier == key:
        flight_carrier_str = value

In [358]:
# crafting response for user
i = 1
print(f'Flight Offer {i}: Toronto - {locations[0]} -> New York - {locations[1]}')
print(f'Departing at: {de_time}')
print(f'Arriving at: {re_time}')
print(f'Trip Duration: {trip_duration}')
print(f'Flight Carrier: {flight_carrier_str}')
print(f'{ticket_avail} tickets left')

Flight Offer 1: Toronto - YYZ -> New York - JFK
Departing at: 2020-08-01T10:15:00-04:00
Arriving at: 2020-08-01T11:49:00-04:00
Trip Duration: 0DT1H34M
Flight Carrier: DELTA AIR LINES
9 tickets left
